In [ ]:
# Install required packages
!pip install gradio_client requests

In [ ]:
import os
import time
import requests
from gradio_client import Client, handle_file
from pathlib import Path

## Configuration

Set the server URL and your audiobook generation parameters here.

In [ ]:
# =============================================================================
# CONFIGURATION - Modify these values for your setup
# =============================================================================

# Server URL - change this to your Audiobook Creator server address
GRADIO_SERVER_URL = "http://localhost:7860"  # e.g., "http://192.168.1.100:7860"

# Path to your EPUB file
EPUB_FILE_PATH = "/path/to/your/book.epub"

# Book title (optional - will use filename if not specified)
BOOK_TITLE = ""  # Leave empty to auto-detect from filename

# TTS Engine: "Orpheus" or "Chatterbox"
TTS_ENGINE = "Orpheus"

# Voice selection (for Orpheus: tara, zac, jess, leo, mia, zoe, etc.)
NARRATOR_VOICE = "zac"

# Output format: "M4B (Chapters & Cover)" or "mp3" or "wav"
OUTPUT_FORMAT = "M4B (Chapters & Cover)"

# Enable emotion tags (Orpheus only) - adds <laugh>, <sigh>, etc.
ADD_EMOTION_TAGS = False

# Reference audio for Chatterbox voice cloning (only needed if TTS_ENGINE="Chatterbox")
REFERENCE_AUDIO_PATH = None  # e.g., "/path/to/voice_sample.wav"

# Output directory for downloaded audiobooks
OUTPUT_DIRECTORY = "./downloaded_audiobooks"

## Connect to the Gradio Server

In [ ]:
# Initialize the Gradio client
print(f"Connecting to Audiobook Creator at {GRADIO_SERVER_URL}...")
client = Client(GRADIO_SERVER_URL)
print("✅ Connected successfully!")

# View available API endpoints
print("\n📋 Available API endpoints:")
client.view_api()

## Step 1: Extract Text from EPUB

First, we upload the EPUB file and extract the text content.

In [ ]:
def extract_text_from_epub(client, epub_path):
    """
    Upload an EPUB file and extract text.
    
    Args:
        client: Gradio client instance
        epub_path: Path to the EPUB file
        
    Returns:
        str: Extracted text content
    """
    print(f"📚 Uploading and extracting text from: {epub_path}")
    
    # Call the text extraction endpoint
    result = client.predict(
        book_file=handle_file(epub_path),
        api_name="/text_extraction_wrapper"
    )
    
    print(f"✅ Text extraction complete!")
    print(f"   Extracted {len(result.split()) if result else 0} words")
    
    return result

In [ ]:
# Extract text from the EPUB file
extracted_text = extract_text_from_epub(client, EPUB_FILE_PATH)

# Preview the first 500 characters
if extracted_text:
    print("\n📖 Preview of extracted text:")
    print("-" * 50)
    print(extracted_text[:500])
    print("...")

## Step 2: Generate Audiobook

Start the audiobook generation process. This is an async operation that returns a job ID for tracking progress.

In [ ]:
def get_book_title(epub_path, custom_title=""):
    """Get book title from custom input or filename."""
    if custom_title:
        return custom_title
    return Path(epub_path).stem


def generate_audiobook(client, epub_path, tts_engine, voice, output_format, 
                       add_emotion_tags=False, book_title="", reference_audio=None):
    """
    Start audiobook generation.
    
    Args:
        client: Gradio client instance
        epub_path: Path to the EPUB file
        tts_engine: "Orpheus" or "Chatterbox"
        voice: Voice name (e.g., "zac", "tara")
        output_format: "M4B (Chapters & Cover)", "mp3", or "wav"
        add_emotion_tags: Whether to add emotion tags (Orpheus only)
        book_title: Custom book title (optional)
        reference_audio: Path to reference audio for Chatterbox (optional)
        
    Returns:
        tuple: (job_id, initial_status)
    """
    title = get_book_title(epub_path, book_title)
    print(f"🎧 Starting audiobook generation...")
    print(f"   Title: {title}")
    print(f"   TTS Engine: {tts_engine}")
    print(f"   Voice: {voice}")
    print(f"   Format: {output_format}")
    print(f"   Emotion Tags: {add_emotion_tags}")
    
    # Prepare reference audio if using Chatterbox
    ref_audio = handle_file(reference_audio) if reference_audio else None
    
    # Submit the generation job
    job = client.submit(
        tts_engine=tts_engine,
        narrator_voice=voice,
        output_format=output_format,
        book_file=handle_file(epub_path),
        add_emotion_tags_checkbox=add_emotion_tags,
        book_title=title,
        reference_audio=ref_audio,
        api_name="/generate_audiobook_wrapper"
    )
    
    # Get initial response to extract job_id
    print("\n⏳ Job submitted, waiting for initial response...")
    
    job_id = None
    for output in job:
        if isinstance(output, tuple) and len(output) >= 3:
            # output format: (progress_message, file_path, job_id)
            if output[2]:  # job_id is the third element
                job_id = output[2]
                print(f"\n✅ Job created with ID: {job_id}")
                break
        print(f"   {output}")
    
    return job_id, job

In [ ]:
# Start audiobook generation
job_id, job_handle = generate_audiobook(
    client=client,
    epub_path=EPUB_FILE_PATH,
    tts_engine=TTS_ENGINE,
    voice=NARRATOR_VOICE,
    output_format=OUTPUT_FORMAT,
    add_emotion_tags=ADD_EMOTION_TAGS,
    book_title=BOOK_TITLE,
    reference_audio=REFERENCE_AUDIO_PATH
)

print(f"\n📝 Save this Job ID for tracking: {job_id}")

## Step 3: Poll Job Status

Check the status of a running job using its job ID.

In [ ]:
def get_job_status(client, job_id):
    """
    Get the current status of a job.
    
    Args:
        client: Gradio client instance
        job_id: The job ID to check
        
    Returns:
        dict: Job details including status, progress, output_file
    """
    result = client.predict(
        job_id=job_id,
        api_name="/get_job_details"
    )
    
    return result


def parse_job_status(status_text):
    """
    Parse the job status text into structured data.
    
    Args:
        status_text: Raw status text from the API
        
    Returns:
        dict: Parsed status information
    """
    status = {
        "raw": status_text,
        "is_completed": False,
        "is_failed": False,
        "is_in_progress": False,
        "output_file": None
    }
    
    if "Status: completed" in status_text.lower() or "✅" in status_text:
        status["is_completed"] = True
    elif "Status: failed" in status_text.lower() or "❌" in status_text:
        status["is_failed"] = True
    elif "Status: in_progress" in status_text.lower() or "in progress" in status_text.lower():
        status["is_in_progress"] = True
    
    # Extract output file path if present
    if "Output File:" in status_text:
        lines = status_text.split("\n")
        for line in lines:
            if "Output File:" in line:
                status["output_file"] = line.split("Output File:")[1].strip()
                break
    
    return status

In [ ]:
# Check job status (you can run this cell multiple times to poll)
# Replace with your job ID if checking a different job
CHECK_JOB_ID = job_id  # or manually set: "abc12345"

if CHECK_JOB_ID:
    status_text = get_job_status(client, CHECK_JOB_ID)
    status = parse_job_status(status_text)
    
    print(f"📊 Job Status for {CHECK_JOB_ID}:")
    print("=" * 50)
    print(status_text)
    print("=" * 50)
    
    if status["is_completed"]:
        print("\n🎉 Job completed successfully!")
    elif status["is_failed"]:
        print("\n❌ Job failed!")
    elif status["is_in_progress"]:
        print("\n⏳ Job still in progress...")
else:
    print("❌ No job ID available. Please run the generation step first.")

In [ ]:
def wait_for_job_completion(client, job_id, poll_interval=30, max_wait_time=7200):
    """
    Wait for a job to complete, polling at regular intervals.
    
    Args:
        client: Gradio client instance
        job_id: The job ID to monitor
        poll_interval: Seconds between status checks (default: 30)
        max_wait_time: Maximum seconds to wait (default: 7200 = 2 hours)
        
    Returns:
        dict: Final job status
    """
    start_time = time.time()
    last_progress = ""
    
    print(f"⏳ Waiting for job {job_id} to complete...")
    print(f"   Polling every {poll_interval} seconds (max wait: {max_wait_time}s)")
    print("=" * 50)
    
    while True:
        elapsed = time.time() - start_time
        
        if elapsed > max_wait_time:
            print(f"\n⚠️ Timeout: Job did not complete within {max_wait_time} seconds")
            return None
        
        status_text = get_job_status(client, job_id)
        status = parse_job_status(status_text)
        
        # Print progress update if changed
        if status_text != last_progress:
            elapsed_min = int(elapsed // 60)
            elapsed_sec = int(elapsed % 60)
            print(f"\n[{elapsed_min:02d}:{elapsed_sec:02d}] Progress update:")
            # Print just the progress line, not the full status
            for line in status_text.split("\n"):
                if "Progress:" in line or "Status:" in line:
                    print(f"   {line.strip()}")
            last_progress = status_text
        
        if status["is_completed"]:
            print(f"\n🎉 Job completed in {int(elapsed//60)}m {int(elapsed%60)}s!")
            return status
        
        if status["is_failed"]:
            print(f"\n❌ Job failed after {int(elapsed//60)}m {int(elapsed%60)}s")
            return status
        
        time.sleep(poll_interval)

In [ ]:
# Wait for the job to complete (optional - run this if you want to wait)
# This will poll every 30 seconds until the job completes or fails

if job_id:
    final_status = wait_for_job_completion(
        client=client,
        job_id=job_id,
        poll_interval=30,  # Check every 30 seconds
        max_wait_time=7200  # Wait up to 2 hours
    )
else:
    print("❌ No job ID available. Please run the generation step first.")

## Step 4: Download the Generated Audiobook

Once the job is complete, download the audiobook file.

In [ ]:
def download_audiobook(server_url, job_id, output_dir="./downloaded_audiobooks"):
    """
    Download the generated audiobook file.
    
    Args:
        server_url: Base URL of the Gradio server
        job_id: The job ID of the completed job
        output_dir: Directory to save the downloaded file
        
    Returns:
        str: Path to the downloaded file, or None if failed
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # First, get job details to find the output file
    status_text = get_job_status(client, job_id)
    status = parse_job_status(status_text)
    
    if not status["is_completed"]:
        print(f"❌ Job {job_id} is not completed yet. Current status:")
        print(status_text)
        return None
    
    if not status["output_file"]:
        print(f"❌ No output file found for job {job_id}")
        return None
    
    output_file = status["output_file"]
    filename = os.path.basename(output_file)
    
    # Construct the file download URL
    # Gradio serves files from the /file= endpoint
    file_url = f"{server_url}/file={output_file}"
    
    print(f"📥 Downloading: {filename}")
    print(f"   From: {file_url}")
    
    try:
        response = requests.get(file_url, stream=True)
        response.raise_for_status()
        
        local_path = os.path.join(output_dir, filename)
        total_size = int(response.headers.get('content-length', 0))
        
        with open(local_path, 'wb') as f:
            downloaded = 0
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
                downloaded += len(chunk)
                if total_size > 0:
                    progress = (downloaded / total_size) * 100
                    print(f"\r   Progress: {progress:.1f}% ({downloaded / 1024 / 1024:.1f} MB)", end="")
        
        print(f"\n\n✅ Downloaded successfully to: {local_path}")
        print(f"   File size: {os.path.getsize(local_path) / 1024 / 1024:.2f} MB")
        
        return local_path
        
    except requests.exceptions.RequestException as e:
        print(f"\n❌ Download failed: {e}")
        return None

In [ ]:
# Download the audiobook
DOWNLOAD_JOB_ID = job_id  # or manually set: "abc12345"

if DOWNLOAD_JOB_ID:
    downloaded_file = download_audiobook(
        server_url=GRADIO_SERVER_URL,
        job_id=DOWNLOAD_JOB_ID,
        output_dir=OUTPUT_DIRECTORY
    )
    
    if downloaded_file:
        print(f"\n🎧 Your audiobook is ready: {downloaded_file}")
else:
    print("❌ No job ID available. Please run the generation step first.")

## Utility: List All Jobs

View all jobs on the server (useful for finding old job IDs).

In [ ]:
def list_all_jobs(client):
    """
    Get a list of all jobs on the server.
    
    Args:
        client: Gradio client instance
        
    Returns:
        DataFrame-like result with job information
    """
    result = client.predict(api_name="/refresh_jobs")
    return result


# List all jobs
print("📋 All Jobs on Server:")
print("=" * 80)
jobs_data = list_all_jobs(client)
print(jobs_data)

## Quick Reference: All-in-One Function

A convenience function that handles the entire workflow.

In [ ]:
def create_audiobook_from_epub(
    server_url,
    epub_path,
    tts_engine="Orpheus",
    voice="zac",
    output_format="M4B (Chapters & Cover)",
    add_emotion_tags=False,
    book_title="",
    reference_audio=None,
    output_dir="./downloaded_audiobooks",
    wait_for_completion=True,
    poll_interval=30
):
    """
    Complete workflow: Upload EPUB -> Generate Audiobook -> Download Result
    
    Args:
        server_url: Gradio server URL
        epub_path: Path to EPUB file
        tts_engine: "Orpheus" or "Chatterbox"
        voice: Voice name
        output_format: Output format
        add_emotion_tags: Add emotion tags (Orpheus only)
        book_title: Custom title (optional)
        reference_audio: Reference audio for Chatterbox (optional)
        output_dir: Where to save the downloaded file
        wait_for_completion: Whether to wait for job to complete
        poll_interval: Seconds between status checks
        
    Returns:
        dict: Result containing job_id and downloaded_file path
    """
    result = {
        "job_id": None,
        "downloaded_file": None,
        "status": None
    }
    
    # Connect to server
    print(f"🔌 Connecting to {server_url}...")
    client = Client(server_url)
    print("✅ Connected!\n")
    
    # Extract text
    print("📚 Step 1: Extracting text from EPUB...")
    extract_text_from_epub(client, epub_path)
    print()
    
    # Generate audiobook
    print("🎧 Step 2: Starting audiobook generation...")
    job_id, _ = generate_audiobook(
        client, epub_path, tts_engine, voice, output_format,
        add_emotion_tags, book_title, reference_audio
    )
    result["job_id"] = job_id
    print()
    
    if wait_for_completion and job_id:
        # Wait for completion
        print("⏳ Step 3: Waiting for completion...")
        status = wait_for_job_completion(client, job_id, poll_interval)
        result["status"] = status
        print()
        
        if status and status["is_completed"]:
            # Download the file
            print("📥 Step 4: Downloading audiobook...")
            downloaded_file = download_audiobook(server_url, job_id, output_dir)
            result["downloaded_file"] = downloaded_file
    
    return result

In [ ]:
# Example usage of the all-in-one function
# Uncomment and modify to use:

# result = create_audiobook_from_epub(
#     server_url="http://192.168.1.100:7860",
#     epub_path="/path/to/your/book.epub",
#     tts_engine="Orpheus",
#     voice="zac",
#     output_format="M4B (Chapters & Cover)",
#     add_emotion_tags=False,
#     output_dir="./my_audiobooks"
# )
# 
# print(f"\n🎉 Done! Job ID: {result['job_id']}")
# print(f"   Audiobook: {result['downloaded_file']}")

## Troubleshooting

### Common Issues:

1. **Connection refused**: Make sure the Audiobook Creator server is running and accessible
2. **Timeout errors**: Audiobook generation can take a long time (hours for long books)
3. **Job not found**: Jobs expire after 24 hours
4. **Download fails**: The file may have been cleaned up or the path is incorrect

### Server Status Check:

In [ ]:
# Check if the server is accessible
try:
    response = requests.get(f"{GRADIO_SERVER_URL}/gradio_api/app_id", timeout=5)
    if response.status_code == 200:
        print(f"✅ Server is accessible at {GRADIO_SERVER_URL}")
    else:
        print(f"⚠️ Server returned status code: {response.status_code}")
except requests.exceptions.ConnectionError:
    print(f"❌ Cannot connect to server at {GRADIO_SERVER_URL}")
    print("   Make sure the Audiobook Creator is running.")
except requests.exceptions.Timeout:
    print(f"⚠️ Server at {GRADIO_SERVER_URL} is not responding (timeout)")